In [ ]:
import os
import time
import requests
try:
    import pyarrow
except:
    %pip install pyarrow
    import pyarrow
try:
    import bs4
    from bs4 import BeautifulSoup
except:
    %pip install beautifulsoup4
    import bs4
    from bs4 import BeautifulSoup
try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

In [51]:
headers = {
        "User-Agent": "Mozilla/5.0 AppleWebKit/605.1.15 Version/17.4.1 Safari/605.1.15",
    #    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
        "Accept-Language": "fr-FR,fr;q=0.8",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

In [52]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",  # Updated
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
    "Accept-Language": "fr-FR,fr;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
}

# Optionally remove an existing header
del headers["Sec-Fetch-Dest"]

print(headers)


{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8', 'Accept-Language': 'fr-FR,fr;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'none', 'Sec-Fetch-User': '?1', 'Cache-Control': 'max-age=0'}


In [53]:
def parse_soup(soup: bs4.BeautifulSoup):
    competences = []
    annonce = dict()

    annonce["intitule_poste"] = soup.find("span", {"itemprop": "title"}).text
    annonce["description"] = soup.find("div", {"itemprop": "description"}).text
    annonce["experience"] = soup.find("span", {"itemprop": "experienceRequirements"}).text

    skill = soup.select("ul.skill-list.list-unstyled > li")
    for item in skill:
        exp = item.find_all("span", {"itemprop": "experienceRequirements"})
        educ = item.find_all("span", {"itemprop": "educationRequirements"})
        comp = item.find("span", {"class": "skill skill-competence"})
        sav = item.find("span", {"class": "skill skill-savoir"})
        lang = item.find("span", {"class": "skill skill-langue"})
        perm = item.find("span", {"class": "skill skill-permis"})
        if len(exp) != 0:
            experience = exp[0].text
            exp_requis = True if item.find("span", {"class": "skill-required"}) else False
            annonce["experience"] = (experience, exp_requis)
        if educ:
            education = educ[0].text
            educ_requis = True if item.find("span", {"class": "skill-required"}) else False
            annonce["education"] = (education, educ_requis)
        if lang:
            langue = lang.find("span", {"class": "skill-name"}).text
            langue_requis = True if item.find("span", {"class": "skill-required"}) else False
            competences.append((langue, langue_requis))
        if perm:
            permis = item.find("span", {"class": "skill-name"}).text
            permis_requis = True if item.find("span", {"class": "skill-required"}) else False
            competences.append((permis, permis_requis))
        if comp:
            competence = comp.find("span", {"class": "skill-name"}).text
            requis = True if comp.find("span", {"class": "skill-required"}) else False
            competences.append((competence, requis))
        if sav:
            savoir = sav.find("span", {"class": "skill-name"}).text
            requis = True if sav.find("span", {"class": "skill-required"}) else False
            competences.append((savoir, requis))
    annonce["competences"] = competences
    infos = soup.find("div", {"class": "description-aside col-sm-4 col-md-5"}).select("dd")
    divers = [item.text for item in infos[2:]]
    annonce["divers"] = divers
    return annonce

TODO:
- corriger l'auto-increment de id en cas de conflit

In [162]:
import psycopg2
def to_db(annonce: dict):
    # Connexion à la base de données PostgreSQL
    conn = psycopg2.connect(
        dbname="webmining",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    for comp in annonce["competences"]:
        if isinstance(comp, str):
            competence = comp
            requis = False
            cur.execute("""
                INSERT INTO competences (nom, requis)
                VALUES (%s, %s)
                ON CONFLICT (nom) DO NOTHING;
            """, (competence, requis))
        else:
            # Insertion des données dans la table postes
            # Insert competences (if not already inserted)
            for competence, requis in comp:
                cur.execute("""
                    INSERT INTO competences (nom, requis)
                    VALUES (%s, %s)
                    ON CONFLICT (nom) DO NOTHING;
                """, (competence, requis))

    # Insert postes (job positions)
    cur.execute("""
        INSERT INTO annonces (intitule_poste, description, experience, divers, reference)
        VALUES (%s, %s, %s, %s, %s)
        RETURNING reference;
    """, (annonce["intitule_poste"], annonce["description"], annonce["experience"], annonce["divers"], annonce["reference"]))
    
    # print(cur.fetchone())
    annonce_id = cur.fetchone()[0]
    print(annonce_id)

    for comp in annonce["competences"]:
        if isinstance(comp, str):
            competence = comp
            cur.execute("""
            INSERT INTO annonce_competences (annonce_reference, competence_id)
            SELECT %s, id FROM competences WHERE nom = %s;
            """, (annonce_id, competence))
        else:
            for competence, requis in annonce["competences"]:
                # Insert relationship into the junction table
                cur.execute("""
                    INSERT INTO annonce_competences (annonce_reference, competence_id)
                    SELECT %s, id FROM competences WHERE nom = %s;
                """, (annonce_id, competence))


    # Validation des modifications et fermeture de la connexion
    conn.commit()
    cur.close()
    conn.close()


mots clés:
- data engineer
- data analyst
- data scientist
- big data
- IA
- architecte cloud


In [55]:
try:
    urls = []
    with open("urls.txt", "r") as f:
        urls = [line.replace("\n", "") for line in f]
except:
    urls = []
    offre_FT = [
        "https://candidat.francetravail.fr/offres/recherche?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0",
        "https://candidat.francetravail.fr/offres/recherche?lieux=69381&motsCles=data+analyst&offresPartenaires=true&rayon=30&tri=0",
        "https://candidat.francetravail.fr/offres/recherche?lieux=69381&motsCles=data+scientist&offresPartenaires=true&range=0-19&rayon=30&tri=0",
        "https://candidat.francetravail.fr/offres/recherche?lieux=69381&motsCles=big+data&offresPartenaires=true&rayon=30&tri=0",
        "https://candidat.francetravail.fr/offres/recherche?motsCles=intelligence+artificielle&offresPartenaires=true&rayon=10&tri=0",
        "https://candidat.francetravail.fr/offres/recherche?motsCles=architecte+cloud&offresPartenaires=true&rayon=10&tri=0"]

    for url in offre_FT:
        while True:
            r = requests.get(url, headers= headers)
            if r.status_code != 200:
                print("waiting 5s")
                time.sleep(5)
                continue
            soup = BeautifulSoup(r.text)
            urls = urls + [link.get("href") for link in soup.select('ul[data-container-type="zone"] > li > a')]
            time.sleep(2)
            break
    with open("urls.txt", "w") as f:
        for url in urls:
            f.write(url+ "\n")

In [72]:
len(urls)

224

# Efface toutes les tables

In [ ]:
import psycopg2

def clear_db():
    conn_params = {
        "dbname": "webmining",
        "user": "postgres",
        "password": "postgres",
        "host": "localhost",  # Change to 'my_postgres_container' if using Docker
        "port": "5432"
    }

    drop_tables_sql = """
        DROP TABLE IF EXISTS annonce_competences;
        DROP TABLE IF EXISTS competences;
        DROP TABLE IF EXISTS annonces;
    """
    try:
        with psycopg2.connect(**conn_params) as conn:
            with conn.cursor() as cur:
                cur.execute(drop_tables_sql)
                print("Tables dropped successfully.")
    except Exception as e:
        print("Error dropping tables:", e)



# crée les tables à partir du fichier db_webmining.sql

In [58]:
import psycopg2
def make_table():
    conn = psycopg2.connect(
        dbname="webmining",
        user="postgres",
        password="postgres",
        host="localhost",  # Change if using Docker
        port="5432"
    )

    cur = conn.cursor()


    # Table creation queries
    with open("db_webmining.sql", "r") as f:
        create_tables_sql = f.read()

    cur.execute(create_tables_sql)
    conn.commit()

    print("✅ Tables created successfully!")

    cur.close()
    conn.close()


In [59]:
domaine = "https://candidat.francetravail.fr"

In [60]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "candidat.francetravail.fr",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Sec-GPC": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Chromium";v="134", "Not:A-Brand";v="24", "Brave";v="134"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
}

cookies = {
    "TCPID": "125331445152517760379",
    "dtCookie": "v_4_srv_8_sn_A03B54816D5AF0755CC6317FD915D035_perc_100000_ol_0_mul_1_app-3A6e350f5c3735afd9_1_rcs-3Acss_0",
    "BIGipServerPOOL_PWNOT-00PT28_HTTPS_PN055-VIPA_3_PN055": "2940013066.56131.0000",
    "TS016fc3b0": "0150c672c31d6f6c7a73fc28f33de8c9cb7dc5105bab64f0d4292f3e76519df0c24f2cfae5e0d5cb90ff9f7ca9259859d850aa9eed",
    "JSESSIONID_RECH_OFFRE": "6HqNhHJ-DbDp_Y5VWI_-YG5oCPFoFAzpLXQmdlZ8YZenhh6JuYLm!-292444737",
    "userBadge": "0",
}
session = requests.Session()
session.headers.update(headers)

In [168]:
with open("./urls.txt", "w") as f:
    f.writelines(urls)

In [61]:
domaine + str(urls[i]).replace("\n", "")

'https://candidat.francetravail.fr/offres/recherche/detail/3487041'

In [71]:
clear_db()
make_table()
j=0
for i in range(0,len(urls), 2):
    print(urls[i])
    r = session.get(url=domaine + str(urls[i]).replace("\n", ""), cookies=cookies)
    if r.status_code != 200:
        break
    soup = BeautifulSoup(r.text)  
    annonce = parse_soup(soup)
    annonce["reference"] = urls[i].split("/")[-1]
    to_db(annonce)
    j+=1
    time.sleep(2)

✅ All tables dropped successfully!
✅ Tables created successfully!
/offres/recherche/detail/189KZNQ
189KZNQ
/offres/recherche/detail/189GQNP
189GQNP
/offres/recherche/detail/189GJVT
189GJVT
/offres/recherche/detail/188ZWDF
188ZWDF
/offres/recherche/detail/188ZDGF
188ZDGF
/offres/recherche/detail/188VWGG
188VWGG
/offres/recherche/detail/188SXMM
188SXMM
/offres/recherche/detail/188HLBS
188HLBS
/offres/recherche/detail/188HFWY
188HFWY
/offres/recherche/detail/188GHTY
188GHTY
/offres/recherche/detail/188DDRG
188DDRG
/offres/recherche/detail/187NTTM
187NTTM
/offres/recherche/detail/186RJZQ
186RJZQ
/offres/recherche/detail/189DCYW
189DCYW
/offres/recherche/detail/188RQRZ
188RQRZ
/offres/recherche/detail/189KVTX
189KVTX
/offres/recherche/detail/189KVNN
189KVNN
/offres/recherche/detail/189JDYT
189JDYT
/offres/recherche/detail/189CBPG
189CBPG
/offres/recherche/detail/188ZVFK
188ZVFK
/offres/recherche/detail/189KBNV
189KBNV
/offres/recherche/detail/188QRMV
188QRMV
/offres/recherche/detail/3408437

In [73]:
j

34

# restauration de la bdd depuis les fichiers json individuels

In [ ]:
import json

annonces = []
files = os.listdir("./json")
from itertools import chain



for file in files:
    annonce = {}
    with open("./json/"+file, "r") as f:
        doc = json.load(f)

    annonce["reference"] = doc["reference"]
    annonce["intitule_poste"] = doc["intitule_poste"]
    annonce["description"] = doc["description"]
    annonce["divers"] = " ".join(doc["divers"])
    annonce["experience"] = doc["experience"][0]
    annonce["competences"] = [competence if isinstance(competence, str) else competence[0] for competence in doc["competences"]]
    annonces.append(annonce)

clear_db()
make_table()
for annonce in annonces:
    print(annonce["reference"])
    to_db(annonce)

# Export de la bdd en csv

In [166]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="webmining",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

# Query to get table names
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
""")

tables = [row[0] for row in cursor.fetchall()]

for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f"Table {table} saved to {table}.csv")

# Close connection
cursor.close()
conn.close()

Table annonce_competences saved to annonce_competences.csv
Table competences saved to competences.csv
Table annonces saved to annonces.csv


C:\Users\Joel\AppData\Local\Temp\ipykernel_8620\2618511829.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table}", conn)


# tests